In [1]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import base64
import os
from ipywidgets import interact, Dropdown

# Read data
file = pd.read_excel('GyalsungProject.xlsx', sheet_name='Dzongkhag_Population')
file1 = pd.read_excel('hospital distance.xlsx')
df = pd.read_csv('National Service Project.csv')
std=pd.read_csv('Student data.csv')

# Create subplots for each dzongkhag
unique_dzongkhags = file1['Dzongkhag_Name'].unique()

# Group data by dzongkhag and grade, then sum up the total students
grouped_data = std.groupby(['Dzongkhag_Name', 'Class_Grade'])['Total_students'].sum().reset_index()

# Plotting with Plotly Express
fig = px.bar(grouped_data, x='Dzongkhag_Name', y='Total_students', color='Class_Grade',
             title='Total Students Grouped by Dzongkhag and Grade',
             labels={'Total_students': 'Total Students', 'Dzongkhag_Name': 'Dzongkhag'},
             barmode='group')  # Set barmode to 'group' for column bars


# Path to the folder containing images
images_folder = 'output_plots'
image_files = [f for f in os.listdir(images_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Create a Dash web application
app = JupyterDash(__name__)

# Define a list to store html.Div() elements
div_elements = []

#header
div_elements.append(html.Div([
    html.Div([
        html.Img(src='https://desuung.org.bt/wp-content/uploads/2020/11/desu-2.png', style={'height': '150px', 'width': '150px'}),
    ], style={'width': '20%', 'text-align': 'left', 'display': 'inline-block'}),
    
    html.Div([
        html.H1("GYALSUNG NATIONAL PROJECT", style={'text-align': 'center', 'font-size': '3em'}),
    ], style={'width': '60%', 'display': 'inline-block'}),

    html.Div([
        html.Img(src='https://desuung.org.bt/wp-content/uploads/2023/05/DFG-150x150.png', style={'height': '150px', 'width': '150px'}),
    ], style={'width': '20%', 'text-align': 'right', 'display': 'inline-block'}),
], style={'width': '100%', 'padding': '20px'}))

original_value = std[std['Class_Grade'].isin(['XII'])]['Total_students'].sum()
original_value_2025 = std[std['Class_Grade'].isin(['XI'])]['Total_students'].sum()
original_value_2026 = std[std['Class_Grade'].isin(['X'])]['Total_students'].sum()

div_elements.append(html.Div([
    html.Div([
        html.Hr(),
        html.P(f"Total Male Population: 401,185", style={'font-size': '16px', 'font-weight': 'bold'}),
        html.P(f"Total Female Population: 369,185", style={'font-size': '16px', 'font-weight': 'bold'}),
        html.P(f"Total Population: 7,702,79", style={'font-size': '16px', 'font-weight': 'bold'}),
        html.P(f"Eligible Student(2024): {original_value} (+/- 10%: {original_value * 1.10}, {original_value * 0.90})", style={'font-size': '16px', 'font-weight': 'bold'}),
        html.P(f"Eligible Student(2025): {original_value_2025} (+/-10%: {1.1 * original_value_2025}, | {0.9 * original_value_2025})", style={'font-size': '16px', 'font-weight': 'bold'}),
        html.P(f"Eligible Student(2026): {original_value_2026} (+/-10%: {1.1 * original_value_2026}, | {0.9 * original_value_2026})", style={'font-size': '16px', 'font-weight': 'bold'}),
        html.Hr()
    ], className="card-content"),
], className="card"))

# Dropdown for selecting images
image_dropdown_options = [{'label': image_file, 'value': image_file} for image_file in image_files]
div_elements.append(html.Div([
    dcc.Dropdown(
        id='image-dropdown',
        options=image_dropdown_options,
        value=image_files[0],  # Default selected value
        multi=False
    ),
    html.Div(id='selected-image'),
]))

#eligible for dzongkhag
div_elements.append(html.Div([
    dcc.Graph(
        id='total-students-chart',
        figure=fig  # Use the Plotly Express figure created earlier
    )
]))

# Third graph (pie chart) with dropdown
pie_dzongkhag_options = [{'label': dzongkhag, 'value': dzongkhag} for dzongkhag in df['Dzongkhag_Name'].unique()]

div_elements.append(html.Div([
    dcc.Dropdown(
        id='pie-dzongkhag-dropdown',
        options=pie_dzongkhag_options,
        value=df['Dzongkhag_Name'].unique()[0],  # Default selected value
        multi=False
    ),
    html.Img(id='pie-chart', src=''),
]))

@app.callback(
    Output('pie-chart', 'src'),
    [Input('pie-dzongkhag-dropdown', 'value')]
)
def update_pie_chart(selected_dzongkhag):
    dzongkhag_data = df[(df['Dzongkhag_Name'] == selected_dzongkhag) & (df['Level_of_ school'] == 'XII')]

    # Create a long-format DataFrame suitable for pie chart
    pie_data = dzongkhag_data.melt(id_vars='Level_of_ school', value_vars=['Male_student', 'Female_student'],
                                   var_name='Gender', value_name='Number_of_students')

    # Plot a pie chart with male and female segments for level 'XII'
    fig = px.pie(
        pie_data,
        names='Gender',
        values='Number_of_students',
        title=f'Students in {selected_dzongkhag} for Level XII',
        labels={'Gender': 'Gender'}
    )

    # Convert the Plotly figure to HTML image source
    img_data = fig.to_image(format="png")
    img_src = f'data:image/png;base64,{base64.b64encode(img_data).decode()}'

    return img_src

@app.callback(
    Output('selected-image', 'children'),
    [Input('image-dropdown', 'value')]
)
def update_selected_image(selected_image):
    image_path = os.path.join(images_folder, selected_image)
    encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')

    return html.Div([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '100%'}),
    ])


# First graph (bar chart)
div_elements.append(html.Div([
    dcc.Graph(
        id='bar-chart',
        figure={
            'data': [
                {'x': file['Dzongkhag'], 'y': file['Male'], 'type': 'bar', 'name': 'Male'},
                {'x': file['Dzongkhag'], 'y': file['Female'], 'type': 'bar', 'name': 'Female'},
            ],
            'layout': {
                'title': 'Male and Female Distribution in Each Dzongkhag',
                'xaxis': {'title': 'Dzongkhag'},
                'yaxis': {'title': 'Populations'},
            }
        }
    )
]))

# Second graph (subplot) with dropdown
dzongkhag_options = [{'label': dzongkhag, 'value': dzongkhag} for dzongkhag in file1['Dzongkhag_Name'].unique()]

div_elements.append(html.Div([
    dcc.Dropdown(
        id='dzongkhag-dropdown',
        options=dzongkhag_options,
        value=file1['Dzongkhag_Name'].unique()[0],  # Default selected value
        multi=False
    ),
    dcc.Graph(id='bar-chart-dropdown'),
]))

# Calculate hospital count by Dzongkhag
hospital_count_by_dzongkhag = df.groupby('Dzongkhag_Name')['Hospital_Name'].nunique()

# Hospital count bar chart
div_elements.append(html.Div([
    dcc.Graph(
        id='hospital-count-chart',
        figure=px.bar(
            x=hospital_count_by_dzongkhag.index,
            y=hospital_count_by_dzongkhag.values,
            labels={'x': 'Dzongkhag', 'y': 'Number of Hospitals'},
            title='Hospital Count by Dzongkhag'
        )
    )
]))

# Layout of the dashboard
app.layout = html.Div(children=div_elements)


# Fourth graph (ipywidgets interactive plot)
div_elements.append(html.Div([
    html.Hr(),
    html.Div([
        html.Label('Select Dzongkhag:'),
        dcc.Dropdown(
            id='hospital-dzongkhag-dropdown',
            options=[{'label': dzongkhag, 'value': dzongkhag} for dzongkhag in unique_dzongkhags],
            value=unique_dzongkhags[0],  # Default selected value
            multi=False
        ),
    ]),
    dcc.Graph(id='hospital-distance-plot'),
]))

# Layout of the dashboard
app.layout = html.Div(children=div_elements)

# Callbacks
@app.callback(
    Output('bar-chart-dropdown', 'figure'),
    [Input('dzongkhag-dropdown', 'value')]
)
def update_second_graph(selected_dzongkhag):
    dzongkhag_data = file1[file1['Dzongkhag_Name'] == selected_dzongkhag]

    figure = {
        'data': [
            {'x': dzongkhag_data['Gewog'], 'y': dzongkhag_data['Male'], 'type': 'bar', 'name': 'Male'},
            {'x': dzongkhag_data['Gewog'], 'y': dzongkhag_data['Female'], 'type': 'bar', 'name': 'Female'},
        ],
        'layout': {
            'title': f'{selected_dzongkhag} Population by Gewogs',
            'xaxis': {'title': 'Gewogs'},
            'yaxis': {'title': 'Population'},
        }
    }

    return figure

# Callback for ipywidgets interactive plot
@app.callback(
    Output('hospital-distance-plot', 'figure'),
    [Input('hospital-dzongkhag-dropdown', 'value')]
)
def update_hospital_distance_plot(selected_dzongkhag):
    dzongkhag_data = file1[file1['Dzongkhag_Name'] == selected_dzongkhag]

    # Set up figure
    fig = px.bar(
        dzongkhag_data,
        x='Gewog',
        y=['Hospital1_per(KM)', 'Hospital2_per(KM)', 'Hospital3_per(KM)'],
        labels={'value': 'Distance', 'variable': 'Hospital'},
        title=f'Distance between Gewogs and Hospitals in {selected_dzongkhag}',
        barmode='group'  # Display bars side by side
    )

    return fig

# Add some CSS styles to create the card with box shadow
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8040)


C:\Users\dtenz\anaconda3\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

C:\Users\dtenz\anaconda3\Lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



Dash app running on http://127.0.0.1:8040/


In [2]:
# import pandas as pd
# import folium
# import webbrowser

# # Read data from CSV file
# df = pd.read_csv('gewog-hospital location.csv')

# # Create a map centered at a specific location
# mymap = folium.Map(location=[df['Geowog_Latitude'].mean(), df['Geowog_Longitude'].mean()], zoom_start=9)

# # Add markers for gewogs with a slight offset
# offset = 0.0001  # You can adjust this offset based on your preference
# for index, row in df.iterrows():
#     folium.Marker(location=[row['Geowog_Latitude'] + offset, row['Geowog_Longitude'] + offset],
#                   popup=row['Gewog_Name'],
#                   tooltip=row['Gewog_Name'],  # Add tooltip for hover text
#                   icon=folium.Icon(color='green', icon='info-sign'),
#                   label=f'Gewog: {row["Gewog_Name"]} (G-{index})').add_to(mymap)

# # Add markers for hospitals
# for index, row in df.iterrows():
#     folium.Marker(location=[row['Hospital_latitude'], row['Hospital_longitude']],
#                   popup=row['Hospital_Name'],
#                   tooltip=row['Hospital_Name'],  # Add tooltip for hover text
#                   icon=folium.Icon(color='blue', icon='times'),
#                   label=f'Hospital: {row["Hospital_Name"]} (H-{index})').add_to(mymap)

# # Add layer control to toggle between hospital and gewog markers
# folium.LayerControl().add_to(mymap)

# # Save the map as an HTML file
# mymap.save('map.html')

# # Open the map in the default web browser
# webbrowser.open('map.html', new=2)
